In [1]:
import pandas as pd
import numpy as np

from itertools import chain

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import bokeh.plotting as plt
from bokeh.layouts import gridplot
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def lc480_importer(ct_file):
    '''
    Imports excel or csv files output from a LightCycler 480
    for analysis.
    
    Param
    _____
    
    ct_file : str
        Path to qPCR data file
        
    Returns
    _______
    
    output : a dataframe
        a Pandas DataFrame containing the imported qPCR data
        
    '''
    if ct_file[-4:] == 'xlsx':
        return pd.read_excel(ct_file, 
                             header=1, 
                             usecols=['Pos','Cp'],
                             sep='\t')
    else:
        return pd.read_csv(ct_file, 
                             header=1, 
                             usecols=['Pos','Cp'],
                             sep='\t')

In [3]:
# Sample namer
def namer(ct_file,
         primers,
         samples,
         reps, 
         config='line',
         **kwargs
         ):
    
    '''
    Imports qPCR data and adds sample and primer names. Additionally, adds
    dilutions if required. The output of namer() is used in all subsequent
    analysis tools.
    
    Params
    ______
    
    ct_file : str
        Path to a CSV or Excel file containing the qPCR data. Currently only
        data output from a Lightcycler 480 is supported, but the structure of 
        namer() allows for other importers to be written without disrupting the
        rest of the function.
        
    primers : list of strings
        A list, in order, of the primers. See documentation for supported plate
        arrangements.
        
    samples : list of strings
        A list, in order, or the sample names. See documentation for supported
        plate arrangements.
        
    reps : int
        Number of replicate wells. 2, 3, or 4.
        
    config : str
        A description of how the samples are arranged: 'square' or 'line'. See
        documentation for additional details. Default 'line'
        
    **kwargs : dictionary
        
        with_dil : list of strings
            List of names of samples that have dilution curves.
            
        dil_series : list of ints
            List of dilution factors in order on plate. Dilutions
            should be entered as integers (e.g. a 1:10 dilution 
            should be entered as 10).
            
        dil_rest : int or None
            The dilution of samples that do not have a dilution 
            series. If None, with_dil should contain all samples.
    '''
    
    # Check for valid replicate number
    valid_reps = [2,3,4]
    
    if reps in valid_reps:
        pass
    else:
        raise ValueError('Accepted replicate numbers are 2, 3, and 4.')
    
    
    # Check for invalid conformation
    if reps == 2 and config == 'square':
        ask = input('''You have entered only two technical replicates, 
        but have asserted they are arranged in a square. Would you like to: 
        \n 1) Proceed with line option setting
        \n 2) Cancel analysis and correct replicate number \n''')
        
        if ask == '1':
            config = 'line'
            print('\n Proceeding with line')
        elif ask == '2':
            return print('\n You have chosen to cancel. Please correct your technical replicates.')
        else:
            raise ValueError('''Please enter '1' or '2'.''')
            
    else:
        pass
    
    # Handle dilutions
    if kwargs:
        update_ls = []
        
        for i in samples:
            if i in kwargs['with_dil']:
                for j in kwargs['dil_series']:
                    update_ls.append('_'.join([i,str(j)]))
                            
            else:
                update_ls.append('_'.join([i,str(kwargs['dil_rest'])]))
        
        samples = update_ls
        print(samples)
        
    else:
        pass
    
    # Read in the data
    ct_data = lc480_importer(ct_file)
    
    # Drop empty wells
    ct_data.dropna(inplace=True, ignore_index=True)
    
    # Check that no wells were erroneously removed
    totalwells = len(primers) * len(samples) * reps
    
    if len(ct_data) == totalwells:
        pass
    else:
        raise Exception('''The total number of wells indicated differs from thenumber of wells with Ct values. This usually indicates that some wells did not produce enough PCR product to measure. 
        
        If you wish to proceed analyzing anyway, set the errant NaN values to "exclude". This will flag the wells for removal after naming is completed. Note that excluding entire primer-sample pairs may raise errors in downstreamanalyses.''')
    
    # Primer list (same regardless of configuration)
    primls = []
        
    for p in primers:
        for r in range(reps * len(samples)):
            primls.append(p)
            
    ct_data['Primer'] = primls
    
    # Line configuration
    if config == 'line':
        
        # Sample list
        names = []
        
        for s in samples:
            for r in range(reps):
                names.append(s)
                
        names = names * len(primers)
        
        ct_data['Name'] = names
     
    # Square configuration
    elif config == 'square':
        
        if reps == 3:
            names = list(np.zeros(len(samples)*reps))

            for i,s in enumerate(samples):
                z = 2*i
                s_inds = [z, z+1, z+2*len(samples)-i]

                for ind in s_inds:
                    names[ind] = s

            names = names * len(primers)
            
            ct_data['Name'] = names
            
        elif reps == 4:
            
           # Sample list
            names = []

            for s in samples:
                for r in range(2):
                    names.append(s)

            names = names * len(primers) * 2

            ct_data['Name'] = names 
            
    else:
        raise Exception('Unrecognized config. Choose "line" or "square".')
        
    ct_data['NamePrim'] = ct_data['Name'] + ct_data['Primer']
    
    ct_data = ct_data[ct_data['Cp'] != 'exclude']
    ct_data = ct_data.astype({'Cp':float})
        
    return ct_data

In [4]:
def averager(ct_data):
    '''
    Computes average Ct values and errors for sample-primer pairs
    from namer output
    
    Params
    ______
    
    ct_data : a dataframe
        Output of namer().
        
    Returns
    _______
    
    output : a dataframe
        A Pandas Dataframe containing average Ct values and standard deviation
        for sample-primer pairs  
    
    '''
    # Save unique sample-primer pairs
    uniques = np.unique(ct_data['NamePrim'])
    
    avgdf = pd.DataFrame(np.zeros([len(uniques),5]),
                        columns=['Primer','Name','NamePrim','AvgCt','StdCt'])
    
    # Iterate through primer-sample pairs to get averages
    for i, u in enumerate(uniques):
        # Subset dataframe
        subdf = ct_data[ct_data['NamePrim'] == u]
        
        # Update sample ID information on average dataframe
        avgdf.loc[i,'Primer'] = subdf.iloc[0]['Primer']
        avgdf.loc[i,'Name'] = subdf.iloc[0]['Name']
        avgdf.loc[i,'NamePrim'] = subdf.iloc[0]['NamePrim']
        
        # Calculate averages
        avgdf.loc[i,'AvgCt'] = np.mean(subdf['Cp'])
        avgdf.loc[i,'StdCt'] = np.std(subdf['Cp'])
        
    return avgdf

In [5]:
def avg_filt(ct_data,
             thresh=0.1):
    '''
    Runs averager and removes divergent replicate wells. Wells are removed 
    until each sample-primer pair has a standard deviation less than or equal
    to a user-specified threshold.
    
    Params
    ______
    
    ct_data : a dataframe
        Output of namer().
        
    thresh : float
        Highest acceptable standard deviation for a set of sample-primer
        replicate wells. Default 0.1
        
    Returns
    _______
    
    output : a dataframe
        A Pandas Dataframe containing average Ct values and standard deviation
        for sample-primer pairs. Identical to the output of averager with
        replicates removed.
        
    droppedWells.txt : a text file
        A text file containing the number of replicates dropped for each sample
        -primer pair.
        
    '''
    
    x = 0
    dropped = []
    
    while x == 0:
        avg = averager(ct_data)
        
        if thresh == None:
            x += 1
            
        elif all([i <= thresh for i in avg['StdCt']]):
            x += 1
            
        else:
            avg = avg[avg['StdCt'] > thresh]['NamePrim']
            subdf = ct_data[ct_data['NamePrim'].isin(avg)].copy()
            subdf.reset_index(inplace=True)
            
            nameprim = np.unique(subdf['NamePrim'])
            dropped.append(nameprim)
            
            for n in nameprim:
                filtdf = subdf[subdf['NamePrim'] == n]
       
                arr = np.array(filtdf['Cp'])
                arrsize = len(filtdf)
                subtract = np.zeros((arrsize,arrsize))
                
                
                for i,v in enumerate(arr):
                    subtract[i] = arr - v
                    
                subtract = np.sum(subtract,1)
                subtract = np.abs(subtract)
                subtract = np.argmax(subtract)
                subtract = filtdf.iloc[subtract]['index']
                
                ct_data = ct_data.drop(subtract)
    
    if len(dropped) == 0:
        pass
    else:
        dropped = list(chain.from_iterable(dropped))
        drop_txt = []

        for i in dropped:
            drop_txt.append(i + ' - ' + str(dropped.count(i)) + '\n')
            
        with open('droppedWells.txt', 'w') as f:
             f.writelines(drop_txt)
                
    return avg

In [6]:
def deltact(ct_data,
            housekeeping,
            primers,
            dilution=None,
            thresh=0.1,
            exp_ctrl=None,
            foldchange=False 
            ):
    '''
    Performs delta Ct, delta delta Ct, and/or fold change analysis on output of
    namer().
    
    Params
    ______
    
    ct_data : a dataframe
        Output of namer().
        
    housekeeping : list
        A list of housekeeping primers.
        
    primers : list
        A list of all primers.
        
    dilution : int or None
        If all samples have the same dilution factor, set to None. If samples
        have different dilution factors, set to the dilution factor of samples
        to be included in the dCt analysis. Default None
        
    thresh : float or None
        Highest acceptable standard deviation of replicate wells. If None, no 
        thresholding is performed. If float, divergent wells are removed until
        replicate wells have acceptable deviation, and the number of removed 
        wells for each sample primer pair is sent to droppedWells.txt. 
        
    exp_ctrl : dictionary or None
        A dictionary containing pairs of experimental and control samples for
        ddCt analysis, e.g. {'+Drug1':'DMSO','+Drug2':'DMSO'}. If None, only
        dCt will be returned.
        
    foldchange : Bool
        Whether to convert ddCt values to fold change. Requires a valid
        dictionary for exp_ctrl.
        
    Returns
    _______
    
    output : a dataframe
        A Pandas DataFrame containing the specified calculations.
    
    '''
    # Check for dilution
    if dilution == False:
        pass
    else:
        ct_data['Dilutions'] = [int(i.split('_')[-1]) for i in ct_data['Name']]
        ct_data = ct_data[ct_data['Dilutions'] == dilution.dil_rest]
        
    # Check incompatible settings 
    if exp_ctrl == None and foldchange == True:
        raise Exception('''Fold change is calculated from ddCt. To calculate fold change, "exp_ctrl" must hold a dictionary.''')
    else:
        pass
    
    # Compute averages 
    avgdf = avg_filt(ct_data,thresh=thresh)
        
    # Check for appropriate housekeeping controls
    if len(housekeeping) == 1:
        ask = input('''Warning: Using only one housekeeping gene severely limits result accuracy.
                    \n Do you want to proceed? [Y/N]''')
        
        if ask == 'Y':
            print('Proceeding with delta delta Ct analysis.')
            
        elif ask == 'N':
            return print('Analysis canceled.')
        
        else:
            raise ValueError('Please enter Y or N to the warning prompt.')
        
    else:
        pass
    
    
    # Batch together housekeeping genes
    names = np.unique(avgdf['Name'])
    
    for n in names:
        subdf = avgdf[avgdf['Name']==n]
        subdf = subdf[subdf['Primer'].isin(housekeeping)]

        errorprop = 0.5 * np.sqrt(np.sum([i**2 for i in subdf['StdCt']]))

        series = pd.Series({'Name':n,
                           'Primer':'housekeeping',
                           'AvgCt':np.mean(subdf['AvgCt']),
                           'StdCt':errorprop
                               })

        avgdf = pd.concat([avgdf,series.to_frame().T],ignore_index=True)
        
    # Calculate delta Ct values (experimental - housekeeping)
    exp_genes = len(primers) - len(housekeeping)
    
    # Make dictionary of empty lists
    dct_dict = {'Name':[],
               'Primer':[],
               'dCt':[],
               'StdErr':[]}
    
    for i, n in enumerate(names):
        # Subset df and make primer loc-able
        subdf = avgdf[avgdf['Name']==n]
        subdf.set_index('Primer',inplace=True)
        
        for p in primers:
            if p in housekeeping:
                pass
            else:
                # Calculate dCt and propagate error
                dct = subdf.loc[p,'AvgCt'] - subdf.loc['housekeeping','AvgCt']
                err = np.sqrt(subdf.loc[p,'StdCt']**2 + subdf.loc['housekeeping','StdCt']**2)
                
                # Update dictionary
                dct_dict['Name'].append(n)
                dct_dict['Primer'].append(p)
                dct_dict['dCt'].append(dct)
                dct_dict['StdErr'].append(err)
                
    # Convert dictionary to dataframe
    dct_df = pd.DataFrame(dct_dict)
    
    if exp_ctrl == None:
        return dct_df
    else:
        pass
    
    # Set index to nameprim for easy location
    dct_df['NamePrim'] = dct_df['Name'] + dct_df['Primer']
    dct_df.set_index('NamePrim',inplace=True)
    
    # Collect the names of the primers used
    dct_prims = np.unique(dct_df['Primer'])
    
    # Make dictionary of empty lists
    ddct_dict = {'Experimental':[],
                'Control':[],
                'Primer':[],
                'Exp dCt':[],
                'ddCt':[],
                'StdErr':[]}
    
    # Calculate delta delta Ct values (experimental dCt - control dCt)
    for e in exp_ctrl:
        # Identify control sample
        c = exp_ctrl[e]
        
        # Loop through primers
        for p in dct_prims:
            # Identify indices
            e_prim = e + p
            c_prim = c + p
            
            # Calculate ddCt and propagate error
            ddct = dct_df.loc[e_prim,'dCt'] - dct_df.loc[c_prim,'dCt']
            err = np.sqrt(dct_df.loc[e_prim,'StdErr']**2 + dct_df.loc[c_prim,'StdErr']**2)
            
            # Update dictionary
            ddct_dict['Experimental'].append(e)
            ddct_dict['Control'].append(c)
            ddct_dict['Primer'].append(p)
            ddct_dict['Exp dCt'].append(dct_df.loc[e_prim,'dCt'])
            ddct_dict['ddCt'].append(ddct)
            ddct_dict['StdErr'].append(err)
            
    # Convert dictionary to dataframe
    ddct_df = pd.DataFrame(ddct_dict)
    
    # Calculate fold change if specified
    if foldchange == True:
        ddct_df['FoldChange'] = 2**(-1 * ddct_df['ddCt'])
    else:
        pass
    
    # Return dataframe
    return ddct_df

In [37]:
def efficiency(ct_in, # output from namer
              with_dil,
              returnmodel=False, # Whether or not to output the linear model in full
              ):    
    
    '''
    Calculates efficiency of qPCR primers based on a standard curve. Returns
    efficiency values and log-transformed plots of the dilution curves.
    
    Params
    ______
    
    ct_data : a dataframe
        Output of namer().
        
    with_dil : list of strings
        List of samples that have dilution series. Generally has only one value,
        but in specific cases multiple samples may have curves.
        
    dil_series : list of ints
        List of dilution factors in order on the plate. E.g. a dilution series 
        from 1:20 to 1:160 could be entered as [20, 40, 80, 160]. Any dilution
        series is acceptable, but ideally it should cover at least a three
        cycle difference.
        
    returnmodel : Bool
        Whether to return a dataframe containing the linear regression model. 
        Default False
        
    Returns
    _______
    
    plot_dict : a dictionary
        A dictionary in which the keys are samples and the value is a list of
        Bokeh plots for each primer tested.
        
    eff_df : a dataframe
        A dataframe containing the efficiency values and R^2 statistic
        calculated for each sample-primer pair. 
        
    model_df : a dataframe
        Only returned if returnmodel == True. A dataframe containing the
        intercept and slope of the linear model for each sample-primer pair.
            
    '''
    # Copy dataframe
    ct_data = ct_in.copy()
    
    # Generate dilution column and remove from names
    name_ls = [i.split('_') for i in ct_data['Name']]

    ct_data['Dilution'] = [1/int(i[-1]) for i in name_ls]
    ct_data['Name'] = ['_'.join(i[:-1]) for i in name_ls]

    # Filter for samples with dilution curves
    ct_data = ct_data[ct_data['Name'].isin(with_dil)]

    # Calculate log2 dilution
    ct_data['Log2Dil'] = np.log2(ct_data['Dilution'])

    # Initiate dictionaries for efficiency values and plots
    eff_dict = {'Name':[],
               'Primer':[],
               'Efficiency':[],
               'Rsquared':[]}
    
    model_dict = {'Name':[],
                 'Primer':[],
                 'Coefficient':[],
                 'Intercept':[]}

    plot_dict = {}

    # Set linear regression prediction range
    minpred = min(ct_data['Log2Dil'])
    maxpred = max(ct_data['Log2Dil'])

    # Perform linear regression and update plots and efficiencies
    for n in np.unique(ct_data['Name']):
        plot_dict[n] = []
      
        # Subset by name
        subdf = ct_data[ct_data['Name'] == n]
        # Loop through primers
        for p in np.unique(subdf['Primer']):
            
            primdf = subdf[subdf['Primer'] == p]
            predrange = np.linspace(minpred,maxpred,len(primdf))[::-1]

            # Set up plot for ct values and regression line
            plot = plt.figure(title=' '.join([n,p]), height=400, width=400,
                              x_axis_label = 'Log2Dil', y_axis_label='Ct',)

            plot.circle(primdf['Log2Dil'].values, primdf['Cp'].values)

            # Perform linear regression
            eff_ld = np.array(primdf['Log2Dil']).reshape(-1,1)
            eff_cp = np.array(primdf['Cp'])

            reg = LinearRegression()
            reg.fit(eff_ld, eff_cp)

            bf = reg.predict(predrange.reshape(-1,1))

            # Update plots and calculate efficiency
            plot.line(predrange, bf)
            eff = 2**(-1 / reg.coef_) - 1
            eff = round(eff[0],3)

            rsquared = r2_score(np.array(primdf['Cp']), bf.reshape(len(bf)))

            eff_dict['Name'].append(n)
            eff_dict['Primer'].append(p)
            eff_dict['Efficiency'].append(eff)
            eff_dict['Rsquared'].append(rsquared)
            
            model_dict['Name'].append(n)
            model_dict['Primer'].append(p)
            model_dict['Coefficient'].append(round(reg.coef_[0],3))
            model_dict['Intercept'].append(round(reg.intercept_,3))

            plot_dict[n].append(plot)
            
    # Convert dicts to dfs
    eff_df = pd.DataFrame(eff_dict)
    model_df = pd.DataFrame(model_dict)
    
    # Return efficiency values with or without model
    if returnmodel == True:
        return plot_dict, eff_df, model_df
    else:
        return plot_dict, eff_df

In [29]:
df = namer('23.08.15_SHARP-AID-Diff/23.08.15_SHARP-AID-Diff_Ct.csv',
         primers=['18S','Polr2a','Xist','Tsix'],
         samples=['72h','48h','24h','0h'],
         reps=3, 
         config='line',
         )

df.head()

,Pos,Cp,Primer,Name,NamePrim
0,A1,9.37,18S,72h,72h18S
1,A2,9.35,18S,72h,72h18S
2,A3,9.38,18S,72h,72h18S
3,A4,9.09,18S,48h,48h18S
4,A5,9.10,18S,48h,48h18S


In [30]:
avg = avg_filt(df)

avg.head()

,Primer,Name,NamePrim,AvgCt,StdCt
0,18S,0h,0h18S,9.710000,0.010000
1,Polr2a,0h,0hPolr2a,20.700000,0.008165
2,Tsix,0h,0hTsix,26.056667,0.070396
3,Xist,0h,0hXist,28.545000,0.035000
4,18S,24h,24h18S,9.410000,0.035590


In [31]:
deltact(df,
        housekeeping=['18S','Polr2a'], 
        primers=['18S','Polr2a','Xist','Tsix'],
        exp_ctrl={'72h':'0h',
                  '48h':'0h',
                  '24h':'0h'},
        dilution=False, 
        foldchange=True 
        )

,Experimental,Control,Primer,Exp dCt,ddCt,StdErr,FoldChange
0,72h,0h,Tsix,8.165000,-2.686667,0.085781,6.438241
1,72h,0h,Xist,9.066667,-4.273333,0.037823,19.337553
2,48h,0h,Tsix,9.191667,-1.660000,0.077226,3.160165
3,48h,0h,Xist,11.283333,-2.056667,0.036094,4.160240
4,24h,0h,Tsix,9.730000,-1.121667,0.084508,2.175982
5,24h,0h,Xist,13.100000,-0.240000,0.102686,1.180993


In [32]:
p_prims = ['Nanog','Oct4','Myc1','Myc2','Klf4']
p_samps = ['0h','72h']

kwargs = {'with_dil' : ['0h'],
         'dil_series' : [20,40,80,160],
         'dil_rest' : 20}

df = namer('23.08.25_PluriPrimerCurve/23.08.25_PluriPrimerCurve_Ct.csv',
          p_prims,
          p_samps,
          3,
          **kwargs)

df.head()

['0h_20', '0h_40', '0h_80', '0h_160', '72h_20']


,Pos,Cp,Primer,Name,NamePrim
0,A1,16.46,Nanog,0h_20,0h_20Nanog
1,A2,16.39,Nanog,0h_20,0h_20Nanog
2,A3,16.37,Nanog,0h_20,0h_20Nanog
3,A4,16.99,Nanog,0h_40,0h_40Nanog
4,A5,16.75,Nanog,0h_40,0h_40Nanog


In [38]:
eff, plots = efficiency(df, # output from namer
              ['0h'],
              returnmodel=False, # Whether or not to output the linear model in full
              )

In [39]:
bokeh.io.show(gridplot(eff['0h'],ncols=2))

In [35]:
plots

,Name,Primer,Efficiency,Rsquared
0,0h,Klf4,0.735,0.386395
1,0h,Myc1,1.423,0.012419
2,0h,Myc2,1.578,-0.024511
3,0h,Nanog,0.896,0.877690
4,0h,Oct4,1.072,0.205723


In [36]:
averager(df)

,Primer,Name,NamePrim,AvgCt,StdCt
0,Klf4,0h_160,0h_160Klf4,32.220000,1.804384
1,Myc1,0h_160,0h_160Myc1,31.930000,1.740000
2,Myc2,0h_160,0h_160Myc2,32.336667,2.732671
3,Nanog,0h_160,0h_160Nanog,19.510000,0.213542
4,Oct4,0h_160,0h_160Oct4,29.303333,1.126331
5,Klf4,0h_20,0h_20Klf4,28.453333,0.094634
6,Myc1,0h_20,0h_20Myc1,29.570000,1.351074
7,Myc2,0h_20,0h_20Myc2,30.083333,2.282869
8,Nanog,0h_20,0h_20Nanog,16.406667,0.038586
9,Oct4,0h_20,0h_20Oct4,26.453333,1.180546
